In [2]:
#Les importations
import numpy as np
import random
import matplotlib.pyplot as plt

In [3]:
#Un bout de code pour représenter graphiquement une gaussienne : pas utile
mu = 0
sigma=1
nums=[]
for i in range(10000):  
    temp = random.gauss(mu, sigma)  
    nums.append(temp)  
        
# plotting a graph  
plt.hist(nums, bins = 200)  
plt.show() 

<ipython-input-3-a97763f6ec7b>:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [4]:
#La génération des paramètres du générateur de pommes
ptot=0 #le dénominateur, somme des 3 probabilités
mu=[0,0,0] #les 3 moyennes, entre 0 et 10
sigma=[0,0,0] #les 3 écarts types entre 0 et 10
p=[0,0,0] #la probabilité de chaque arbre
for i in range(3):
  mu[i]=random.random()*10
  sigma[i]=random.random()*10
  p[i]=random.random()
  ptot+=p[i]
for i in range(3):
  p[i]=p[i]/ptot
for i in range(3):
  print(mu[i],sigma[i],p[i])

4.352604576385351 3.841968322485223 0.012467210611551831
4.122426233875089 8.584544846554655 0.45486697119187786
7.004612996858683 2.334416485959717 0.5326658181965704


In [5]:
#Le générateur de pommes
def generepomme(mu,sigma,p):
  r=random.random()
  if r<p[0]:
    arbre=0
  elif r<p[0]+p[1]:
    arbre=1
  else: arbre=2 #on a sélectionné l'arbre
  pos=random.gauss(mu[arbre], sigma[arbre]) #on applique l'arbre
  #print(arbre,pos)
  return pos
#On teste
nums=[]
for i in range(1000):
  x=generepomme(mu,sigma,p)
  nums.append(x)
plt.hist(nums, bins = 200)  
plt.show() 

<ipython-input-5-cb69fe292cab>:18: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
nums

In [6]:
def baseline(tirages,nbtests):
  return(random.random()*5,5+random.random()*5)
  #cet algorithme est stupide et fonctionne assez mal! Dans ce cas il n'y a pas de paramètre mais on peut utiliser en particulier l'historique des pommes précédentes !

In [9]:
def dernier_vu(tirages,nb_tests):#une heuristique: mettre le panier là où la dernière pomme est tombée
  if nb_tests>1:
    x=tirages[nb_tests-1]
    y=tirages[nb_tests-2]
  elif nb_tests>0:
    x=tirages[nb_tests-1]
    y=0.3
  else:
    x=0.7
    y=0.3
  return(x,y)

In [10]:
#Protocole de test de la solution
nb_tests=0
nb_corrects=0
tirages=[]
for i in range(3600):
  #x,y=baseline(tirages,nb_tests)#ici remplacer baseline par le nom de la fonction à tester
  x,y=dernier_vu(tirages,nb_tests)
  z=generepomme(mu,sigma,p)
  #print('x',x,'y',y,'z',z, nb_corrects,nb_tests)
  tirages.append(z)
  if abs(x-z)<0.5 or abs(y-z)<0.5: #diamètre du seau=1, donc rayon=0.5
    nb_corrects+=1
  nb_tests+=1
print('Nombre de pommes ramassées, sur 100 tombées :',100*nb_corrects/nb_tests)
  

Nombre de pommes ramassées, sur 100 tombées : 11.833333333333334
